In [1]:
import pickle
import pandas as pd
from src.paths import ROOT
from scipy.stats import spearmanr
from scipy.spatial.distance import pdist, squareform
from src.fmri_rsa_utils import *
from src.utils import *
import numpy as np

In [2]:
#Creating RDMs for Exp features

# experiential features
exp_ftrs = pd.read_csv(ROOT / 'data/exp_features.csv')

# organising exp arrays into a dict
exp_dict = {exp_ftrs.iloc[i,0]: np.array(exp_ftrs.iloc[i,1:].tolist()) for i in range(len(exp_ftrs))}

# distance matrices (1-cosine_similarity)
s2_exp = pd.read_csv(ROOT / 'data/Study2_model_RSMs/Exp48_SOE320_sim_mat.csv')
words_s2 = s2_exp.Word.tolist()

# organising words from study 2 in a dedicated matrix
s2_exp_mat = np.stack([exp_dict[word] for word in words_s2])

# obtaining RDM
exp_rdm2 = get_distance_vec(s2_exp_mat)

set2_cat = pd.read_csv(ROOT / 'data/word_categories_2.csv')
objects = ['animal', 'food', 'tool', 'vehicle']
events = ['negative event', 'sound', 'social event', 'communication']

In [3]:
# Loading brain data into RDM
brain_2 = get_fmri_rdm_study2_aggregated('semantic')

In [4]:
brain_results = open_json(ROOT / 'results/rsa/rsa_brain_set2_top3.json')
exp48_results = open_json(ROOT / 'results/rsa/rsa_exp48_set2_top3.json')

best_layers = open_json(ROOT / 'results/rsa/best_layers.json')

## Contextualised embeddings

Here, we consider embeddings obtained from nouns plugged into neutral sentences.

In [6]:
simcse_cont = pickle.load(open(ROOT / 'results/simcse/simcse_layers_context.pkl', 'rb'))
mcse_cont = pickle.load(open(ROOT / 'results/mcse/mcse_layers_context.pkl', 'rb'))
clap_cont = pickle.load(open(ROOT / 'results/clap/clap_layers_context.pkl', 'rb'))
bert_cont = pickle.load(open(ROOT / 'results/bert/bert_layers_context.pkl', 'rb'))
vbert_cont = pickle.load(open(ROOT / 'results/visualbert/visualbert_layers_context.pkl', 'rb'))
clip_cont = pickle.load(open(ROOT / 'results/clip/clip_layers_context.pkl', 'rb'))

### Full set

In [ ]:
for model_embs, model_name in zip([simcse_cont, mcse_cont, clap_cont, bert_cont, vbert_cont, clip_cont], ['simcse', 'mcse', 'clap', 'bert', 'visualbert', 'clip']):
    avg_mat = average_representations_across_prompts(model_embs)
    avg_mat_top3 = {word: avg_mat[word][np.array(best_layers[model_name]['brain']), :].mean(axis=0) for word in avg_mat}
    brain_rdm = get_distance_vec(np.stack([avg_mat_top3[word] for word in words_s2]))
    brain_rho, brain_pval = spearmanr(brain_rdm, brain_2)

    avg_mat_top3 = {word: avg_mat[word][np.array(best_layers[model_name]['exp48']), :].mean(axis=0) for word in avg_mat}
    exp_rdm = get_distance_vec(np.stack([avg_mat_top3[word] for word in words_s2]))
    exp_rho, exp_pval = spearmanr(exp_rdm2, exp_rdm)
    print(model_name, round(brain_rho, 2),brain_pval, brain_pval<0.05, round(exp_rho, 2), exp_pval<0.05)
    
    brain_results[model_name]['context']['set2'] = brain_rho
    exp48_results[model_name]['context']['set2'] = exp_rho



simcse 0.22 0.0 True 0.52 True
mcse 0.19 0.0 True 0.45 True
clap 0.0 0.6967885380040464 False 0.03 True
bert 0.23 0.0 True 0.53 True
visualbert 0.12 4.795543045982238e-167 True 0.27 True
clip 0.14 1.8609771360823958e-215 True 0.41 True


### Objects

In [8]:
indices = np.where(set2_cat.category.isin(objects))[0]
brain_data = get_fmri_rdm_study2_aggregated_subset('semantic', indices)
exprdm = get_distance_vec(np.stack([exp_dict[word] for word in set2_cat.word[indices]]))

for model_embs, model_name in zip([simcse_cont, mcse_cont, clap_cont, bert_cont, vbert_cont, clip_cont], ['simcse', 'mcse', 'clap', 'bert', 'visualbert', 'clip']):
    avg_mat = average_representations_across_prompts(model_embs)
    avg_mat_top3 = {word: avg_mat[word][np.array(best_layers[model_name]['brain']), :].mean(axis=0) for word in avg_mat}
    brain_rdm = get_distance_vec(np.stack([avg_mat_top3[word] for word in set2_cat.word])[indices, :])
    brain_rho, _ = spearmanr(brain_rdm, brain_data)

    avg_mat_top3 = {word: avg_mat[word][np.array(best_layers[model_name]['exp48']), :].mean(axis=0) for word in avg_mat}
    exp48_rdm = get_distance_vec(np.stack([avg_mat_top3[word] for word in set2_cat.word])[indices, :])
    exp_rho, _ = spearmanr(exprdm, exp48_rdm)
    print(model_name, round(brain_rho, 2), round(exp_rho, 2))
    brain_results[model_name]['context']['objects'] = brain_rho
    exp48_results[model_name]['context']['objects'] = exp_rho



simcse 0.14 0.42
mcse 0.12 0.4
clap 0.05 0.04
bert 0.13 0.43
visualbert 0.12 0.3
clip 0.1 0.39


### Events

In [9]:
indices = np.where(set2_cat.category.isin(events))[0]
brain_data = get_fmri_rdm_study2_aggregated_subset('semantic', indices)
exprdm = get_distance_vec(np.stack([exp_dict[word] for word in set2_cat.word[indices]]))

for model_embs, model_name in zip([simcse_cont, mcse_cont, clap_cont, bert_cont, vbert_cont, clip_cont], ['simcse', 'mcse', 'clap', 'bert', 'visualbert', 'clip']):
    avg_mat = average_representations_across_prompts(model_embs)
    avg_mat_top3 = {word: avg_mat[word][np.array(best_layers[model_name]['brain']), :].mean(axis=0) for word in avg_mat}
    brain_rdm = get_distance_vec(np.stack([avg_mat_top3[word] for word in set2_cat.word])[indices, :])
    brain_rho, _ = spearmanr(brain_rdm, brain_data)

    avg_mat_top3 = {word: avg_mat[word][np.array(best_layers[model_name]['exp48']), :].mean(axis=0) for word in avg_mat}
    exp48_rdm = get_distance_vec(np.stack([avg_mat_top3[word] for word in set2_cat.word])[indices, :])
    exp_rho, _ = spearmanr(exprdm, exp48_rdm)
    print(model_name, round(brain_rho, 2), round(exp_rho, 2))
    brain_results[model_name]['context']['events'] = brain_rho
    exp48_results[model_name]['context']['events'] = exp_rho



simcse 0.22 0.54
mcse 0.19 0.53
clap -0.06 -0.0
bert 0.27 0.52
visualbert 0.08 0.25
clip 0.18 0.48


## Contextualised embeddings caption-like templates

Here, we consider embeddings obtained from nouns plugged into caption-like sentences.

In [11]:
simcse_cont_vis = pickle.load(open(ROOT / 'results/simcse/simcse_layers_visual_context_specific.pkl', 'rb'))
mcse_cont_vis = pickle.load(open(ROOT / 'results/mcse/mcse_layers_visual_context_specific.pkl', 'rb'))
clap_cont_vis = pickle.load(open(ROOT / 'results/clap/clap_layers_visual_context_specific.pkl', 'rb'))
bert_cont_vis = pickle.load(open(ROOT / 'results/bert/bert_layers_visual_context_specific.pkl', 'rb'))
vbert_cont_vis = pickle.load(open(ROOT / 'results/visualbert/visualbert_layers_visual_context_specific.pkl', 'rb'))
clip_cont_vis = pickle.load(open(ROOT / 'results/clip/clip_layers_visual_context_specific.pkl', 'rb'))


### Full set

In [12]:
for model_embs, model_name in zip([simcse_cont_vis, mcse_cont_vis, clap_cont_vis, bert_cont_vis, vbert_cont_vis, clip_cont_vis], ['simcse', 'mcse', 'clap', 'bert', 'visualbert', 'clip']):
    avg_mat = average_representations_across_prompts(model_embs)
    avg_mat_top3 = {word: avg_mat[word][np.array(best_layers[model_name]['brain']), :].mean(axis=0) for word in avg_mat}
    brain_rdm = get_distance_vec(np.stack([avg_mat_top3[word] for word in words_s2]))
    brain_rho, _ = spearmanr(brain_rdm, brain_2)

    avg_mat_top3 = {word: avg_mat[word][np.array(best_layers[model_name]['exp48']), :].mean(axis=0) for word in avg_mat}
    exp_rdm = get_distance_vec(np.stack([avg_mat_top3[word] for word in words_s2]))
    exp_rho, _ = spearmanr(exp_rdm2, exp_rdm)
    print(model_name, round(brain_rho, 2), round(exp_rho, 2))
    brain_results[model_name]['visual context']['set2'] = brain_rho
    exp48_results[model_name]['visual context']['set2'] = exp_rho



simcse 0.23 0.55
mcse 0.2 0.5
clap 0.1 0.22
bert 0.26 0.59
visualbert 0.14 0.3
clip 0.15 0.47


### Objects

In [13]:
indices = np.where(set2_cat.category.isin(objects))[0]
brain_data = get_fmri_rdm_study2_aggregated_subset('semantic', indices)
exprdm = get_distance_vec(np.stack([exp_dict[word] for word in set2_cat.word[indices]]))

for model_embs, model_name in zip([simcse_cont_vis, mcse_cont_vis, clap_cont_vis, bert_cont_vis, vbert_cont_vis, clip_cont_vis], ['simcse', 'mcse', 'clap', 'bert', 'visualbert', 'clip']):
    avg_mat = average_representations_across_prompts(model_embs)
    avg_mat_top3 = {word: avg_mat[word][np.array(best_layers[model_name]['brain']), :].mean(axis=0) for word in avg_mat}
    brain_rdm = get_distance_vec(np.stack([avg_mat_top3[word] for word in set2_cat.word])[indices, :])
    brain_rho, _ = spearmanr(brain_rdm, brain_data)

    avg_mat_top3 = {word: avg_mat[word][np.array(best_layers[model_name]['exp48']), :].mean(axis=0) for word in avg_mat}
    exp_rdm = get_distance_vec(np.stack([avg_mat_top3[word] for word in set2_cat.word])[indices, :])
    exp_rho, _ = spearmanr(exprdm, exp_rdm)
    print(model_name, round(brain_rho, 2), round(exp_rho, 2))
    brain_results[model_name]['visual context']['objects'] = brain_rho
    exp48_results[model_name]['visual context']['objects'] = exp_rho



simcse 0.12 0.46
mcse 0.11 0.42
clap 0.09 0.13
bert 0.14 0.53
visualbert 0.12 0.34
clip 0.1 0.45


### Events

In [14]:
indices = np.where(set2_cat.category.isin(events))[0]
brain_data = get_fmri_rdm_study2_aggregated_subset('semantic', indices)
exprdm = get_distance_vec(np.stack([exp_dict[word] for word in set2_cat.word[indices]]))

for model_embs, model_name in zip([simcse_cont_vis, mcse_cont_vis, clap_cont_vis, bert_cont_vis, vbert_cont_vis, clip_cont_vis], ['simcse', 'mcse', 'clap', 'bert', 'visualbert', 'clip']):
    avg_mat = average_representations_across_prompts(model_embs)
    avg_mat_top3 = {word: avg_mat[word][np.array(best_layers[model_name]['brain']), :].mean(axis=0) for word in avg_mat}
    brain_rdm = get_distance_vec(np.stack([avg_mat_top3[word] for word in set2_cat.word])[indices, :])
    brain_rho, _ = spearmanr(brain_rdm, brain_data)

    avg_mat_top3 = {word: avg_mat[word][np.array(best_layers[model_name]['exp48']), :].mean(axis=0) for word in avg_mat}
    exp_rdm = get_distance_vec(np.stack([avg_mat_top3[word] for word in set2_cat.word])[indices, :])
    exp_rho, _ = spearmanr(exprdm, exp_rdm)
    print(model_name, round(brain_rho, 2), round(exp_rho, 2))
    brain_results[model_name]['visual context']['events'] = brain_rho
    exp48_results[model_name]['visual context']['events'] = exp_rho



simcse 0.23 0.56
mcse 0.21 0.58
clap 0.06 0.21
bert 0.27 0.57
visualbert 0.1 0.27
clip 0.19 0.52


Saving results

In [22]:
dict_to_json(brain_results, ROOT / 'results/rsa/rsa_brain_set2_top3.json')
dict_to_json(exp48_results, ROOT / 'results/rsa/rsa_exp48_set2_top3.json')

## Isolated words

Here, we're considering embeddings obtained by inputting the models with isolated words.

In [15]:
simcse = pickle.load(open(ROOT / 'results/simcse/simcse_layers.pkl', 'rb'))
mcse = pickle.load(open(ROOT / 'results/mcse/mcse_bert_coco_layers.pkl', 'rb'))
clap = pickle.load(open(ROOT / 'results/clap/clap_layers.pkl', 'rb'))
clip = pickle.load(open(ROOT / 'results/clip/clip.pkl', 'rb'))
bert = pickle.load(open(ROOT / 'results/bert/bert_layers.pkl', 'rb'))
vbert = pickle.load(open(ROOT / 'results/visualbert/visualbert_layers.pkl', 'rb'))

In [17]:
brain_results_full = open_json(ROOT / 'results/rsa/rsa_brain_set2.json')
exp48_results_full = open_json(ROOT / 'results/rsa/rsa_exp48_set2.json')

In [19]:
new_best_layers = {}
for model_name in ['simcse', 'mcse','clap', 'bert', 'visualbert']:
    new_best_layers[model_name] = {}
  
    best_model_layers = np.flip(np.argsort(brain_results_full[model_name]['non-contextualised']['set2']))
    new_best_layers[model_name]['brain'] = best_model_layers[:3]

    best_model_layers = np.flip(np.argsort(exp48_results_full[model_name]['non-contextualised']['set2']))
    new_best_layers[model_name]['exp48'] = best_model_layers[:3]
    # print(best_model_layers[:3])


In [20]:
new_best_layers

{'simcse': {'brain': array([11, 10, 12]), 'exp48': array([10,  8,  9])},
 'mcse': {'brain': array([10,  9, 11]), 'exp48': array([ 9, 10,  8])},
 'clap': {'brain': array([ 7,  8, 12]), 'exp48': array([12, 11, 10])},
 'bert': {'brain': array([ 1, 11,  2]), 'exp48': array([ 1,  2, 10])},
 'visualbert': {'brain': array([ 9, 10,  8]), 'exp48': array([ 9,  8, 10])}}

In [21]:
for model_embs, model_name in zip([simcse, mcse, clap, bert, vbert], ['simcse', 'mcse', 'clap', 'bert', 'visualbert']):
    
    avg_mat_top3 = {word: model_embs[word][np.array(new_best_layers[model_name]['brain']), :].mean(axis=0) for word in avg_mat}
    brain_rdm = get_distance_vec(np.stack([avg_mat_top3[word] for word in words_s2]))
    brain_rho, brain_pval = spearmanr(brain_rdm, brain_2)

    avg_mat_top3 = {word: model_embs[word][np.array(new_best_layers[model_name]['exp48']), :].mean(axis=0) for word in avg_mat}
    exp_rdm = get_distance_vec(np.stack([avg_mat_top3[word] for word in words_s2]))
    exp_rho, exp_pval = spearmanr(exp_rdm2, exp_rdm)
    print(model_name, round(brain_rho, 2),brain_pval, brain_pval<0.05, round(exp_rho, 2), exp_pval<0.05)
    brain_results[model_name]['non-contextualised']['set2'] = brain_rho
    exp48_results[model_name]['non-contextualised']['set2'] = exp_rho



simcse 0.18 0.0 True 0.29 True
mcse 0.19 0.0 True 0.35 True
clap 0.03 1.2888811514874077e-08 True 0.01 True
bert 0.05 3.1106962922083e-32 True 0.16 True
visualbert 0.09 8.387507311255873e-86 True 0.17 True
